 # Computational Methods for Linguistic Typology

## Part 2 - The data cycle

Matthew J Carroll

Australian Linguistic Society 2022 CoEDL Masterclasses

<center>
<img src="Images/typscience.jpg" width="1000"> 

# Identifying a base

## Verbose Exponence

|Number|Form|
|-----------|-----------|
|SG| *wɐ-mdəd-ə* |
|DU| *nə-mdəd-anɛ*|
|PL|*nə-mdəd-ə*|

1 PRS Partial Paradigm of yəmdədə 'to sit' in Yei

*Verbose exponence*: Two (or more) co-occurring formatives which both provide *some information about* a feature, bundle of features or derivational category in more than one position in a domain

# Data acquisition

<center>
<img src="Images/typscience.jpg" width="1000"> 

- Gather data
- Exploring data
- Update Data


**How do we know where to look?**

<img src="Images/pacific_central.jpg" width=15000>

- Inferring possible candidate languages from typological knowledge
- Conferring with colleagues

<img src="Images/library.jpg" class="center">

**How can we speed up the grammar reading process?**

- 1 week to sufficiently read a grammar
- 25% are relevant to the general topic
- < 10% of those fill the distant cells of the typology
- 40 weeks to get a single example for 1 cell in the typology

## Data acquistion: gather data

![](Images/babel2.png)

## Data acquisition: explore the data

![](Images/fedden.png)

![](Images/newscan.png)

![](Images/oldscan.png)

![](Images/oldscanbad.png)

![](Images/oldscan3.png)

Different versions of the same document



Filename: Author-TextIDYear-Version.pdf

- adelaar-amalgamation-malagasy2010.pdf
- adelaar-austronesian2013.pdf
- adelaar-austronesian-asia-madagascar2005.pdf
- adelaar-austronesian-asia-madagascar2005v2.pdf
- adelaar-austronesian-historical2005.pdf
- adelaar-austronesian-historical2005-o.pdf

#### Data understanding report

- 10000 > PDFs
    - The 'best' reference for a language
        - Grammars
        - Other descriptive materials
- Majority OCR’d
    - Already searchable
- Varying quality and sources
    - Fairly messy and difficult to access
- Lots of duplicate files
    - Best files have the longest name

## Data acquisition: update the data

Before we can search the data we need to:

Step 1 - Clean up the PDFs
- Sort the collection
- Remove bad candidates for searching

Step 2 - Extract the text
- Existing tool pdftotext.exe


### Step 1 - Clean the data

How do we handle repeated ﬁles?

Filename: Author-TextIDYear-Version.pdf

- adelaar-amalgamation-malagasy2010.pdf
- adelaar-austronesian2013.pdf
- adelaar-austronesian-asia-madagascar2005.pdf
- adelaar-austronesian-asia-madagascar2005v2.pdf
- adelaar-austronesian-historical2005.pdf
- adelaar-austronesian-historical2005-o.pdf

Import the packages we need:

In [ ]:
import os
import re
import fnmatch
from PyPDF2 import PdfFileReader

In [67]:
PyPDF2.__version__

NameError: name 'PyPDF2' is not defined

Our folder containing pdfs:

In [68]:
pdfdir = os.listdir('./grammars/pdfs/')

We want the longest file names:

Find files whose names are a substring of another file:

In [69]:
subsetlst = []

for file in pdfdir:
    filestring = file[0:-4] + '.+' + '\.pdf'
    regex = re.compile(filestring) # <filename>.+\.pdf
    for file2 in pdfdir:
        if regex.match(file2):
            subsetlst.append(file) 

In [70]:
subsetlst2 = []

for file in pdfdir:
    filestring = file[0:-4] 
    for file2 in pdfdir:
        file2string = file2[0:-4]
        if filestring == file2string: 
            pass
        elif filestring in file2string: 
            subsetlst2.append(file)  

Make a list of the files whose names aren't a subset of another:

In [71]:
uniqlst = []     

for file in pdfdir:
    if file not in subsetlst:
        uniqlst.append(file)

In [72]:
print(uniqlst) 

['bada_agusan2011.pdf', 'aikhenvald_sepik2008v2.pdf', 'bastian_papua1885.pdf', 'appell-warren_tarakan-tidung1986.pdf', 'adelaar-pawley_festschrift-blust2009pages.pdf', 'anonymous_nala1892.pdf', 'barlow_murik1985.pdf', 'aufinger_madang1942-1945.pdf', 'ambiaetal_ambulas-wosera-mamu1991.pdf', 'afable_kallahan1989.pdf', 'anonymous_udama1913.pdf', 'alua_nara2013_s.pdf', 'barbosa_agta-lamika1985.pdf', 'balbi_atlas-introduction1826_o.pdf', 'armour_tausug1984_o.pdf', 'anonymous_maipua1895v2.pdf', 'anonymous_purari1915.pdf', 'arsjoe_ama1993_o.pdf', 'bancel-etang_eurasiatic-pronouns2008_s.pdf', 'aikhenvald_manambu2008.pdf', 'barbour_neverver2009.pdf', 'anonymous_kwagila1895v2.pdf', 'ballantyne-jenness_goodenough1926-1928.pdf', 'aufenanger_yuat1977.pdf', 'anceaux_yapen1961_o.pdf', 'arsjo_ama-vowels1980.pdf', 'baarda_inleiding-lodasch1904.pdf', 'barth_fly-headwaters1971v2.pdf', 'abbott_nor-pondo1985.pdf', 'adelaar_malay-malagasy1989.pdf', 'anonymous_nng1910.pdf', 'bastard_karo1920v2.pdf', 'anderse

### Quality analyser





- How do we work out what is a good quality pdf?
- High quality scans ≠ Good quality OCR


<img src="Images/goodscan.png" class="center">

How do we work out what is a good quality pdf?
- Clear text on white pages
- Prefer generated PDFs
- Very clean scans on text mode
- Lower file sizes

pdfqual:

- File size / Number of pages
- Asks for a threshold (20,000 default)
- Prints out a list of the files from the previous list of all pdfs which are < threshold

In [ ]:
threshold = input('Enter quality threshold in bytes (Basic 20000): ')

with open("statfile.tsv", "w") as statfile:
    for file in uniqlst:
        pdf = PdfFileReader('./grammars/pdfs/' + file)
        pages = pdf.getNumPages()
        size = os.stat('./grammars/pdfs/' + file).st_size
        qual = size / pages
        if qual < float(threshold):
            output = str(file) + '\t' + str(qual) + '\n'
            statfile.write(output)


### Step 2 - Scrape the text


`for /F "tokens=1" %%A in (.\statfile.tsv) do .\pdftotext.exe -enc UTF-8 ..\pdfs\%%A .\texts\%%A.txt`


### Gather the data

*Search for the same gloss twice in the one word*

In [ ]:
leipzig = ['1','2','3','1SG','1DU', '1DL','1PL','2SG','2DU','2DL','2PL' '3SG','3DU','3DL','3PL','ABE','ABL','ABS','ACC','ACCOM','ACT','ADJ','ADE','ADM','ADV','AFF','AG','AGT','AGR','ALL','AL','ALLOC','ALIEN','AN','AND','ANT','ANTE','ANTIC','ANTIP','AP','AOR','APP','APL','APPL','APPR','APRX','ART','ASP','ASS','AT','ATT','AUD','AUG','AUX','BEN','CAP','CAU','CAUS','CENT','CF','CL','CNSQ','COL','COM','COMP','COMPL','CPL','CONC','COND','CONJ','CONT','CTN','CNTR','COP','COR','CRAS','CRS','DAT','DE','DEC','DECL','DEF','DEL','DEL','DEO','DEP','DES','DESI','DEST','DIM','DIR','DISJ','DIST','DISTR','DITR','DLM','DU','DUB','DUR','DY','DYAD','DYN','ELA','EMP','EPIS','ERG','ESS','EV','EVID','EVIT','EX','EXCL','EXCLAM','DUR','EXESS','EXH','EXIST','EXO','EXP','EXPER','FEM','FACT','FOC','FORM','FP','FR','FREQ','FUT','GEN','GER','GNO','GT','HAB','HBL','HEST','HIST','HOD','HON','HORT','HSY','HUM','HYP','IGNOR','ILL','IMM','IMP','IMPERF','IMPR','IMPREC','INCH','INCHO','INCEP','IND','INDF','NDEF','INE','INF','INFER','INFR','INEL','INS','INSTR','INT','NTR','INV','IO','IPFV','IRR','IS','ITER','JUS','LAT','LD','LOC','LOG','MASC','MID','MIM','MIR','MLT','MLTP','MOD','MOM','NEUT','NEG','NMZ','NZ','NOMI','NOM','NS','NUM','OBJ','OB','OBL','OBV','OPT','PART','PAS','PASS','PAT','PA','PAU','PEG','PER','PERF','PRF','PERS','PFV','PL','PLU','PLUR','PN','PRO','PO','POL','POS','POSS','POST','POSTE','POSTEL','POT','PP','PPFV','PPP','PR','PREC','PRED','PREP','PRESP','PRET','PRT','PRF','PERF','PRIV','PRS','PRES','PROB','PROG','PROH','PROL','PROP','PROS','PROSP','PRSP','PROT','PROX','PST','PT','PTCP','PCP','PTV','PURP','QUOT','REAL','REC','RECP','REF','RFR','REFL','REL','REM','REP','RES','RET','SBJ','SUB','SBJV','SJV','SE','SEM','SENS','SEQ','SG','SGV','SIM','SJV','SBJV','SPEC','SS','STAT','STV','SUB','SU','SUBR','SUBORD','SBRD','SR','SUBE','SUBL','SUC','SUP','SUPE','TAM','TEL','TEMP','TERM','TNS','TOP','TR','TRANS','TRANSL','TRI','TRN','TVF','UH','UND','UR','USIT','VB','VBZ','VD','VEN','VER','VIA','VIS','VI','VN','VOC','VOL','VT','WH']

In [ ]:
with open("multiple_exponence.tsv", 'w') as results:
    for lang in os.listdir('./grammars/texts/'):
        print(lang) #provides feedback so we can see this
        for gloss in leipzig:
            page = 1
            linenum = 0 
            rex1 = str(gloss) + "[\|\\\.\-][^\d\s]+[\|\\\.\-]" + str(gloss) 
            rex2 = "[^\d][\|\\\.\-]?" + str(gloss) + "\-" + str(gloss) + "[\|\\\.\-]?[^\d]"
            regex = "(" + rex1 + ")|(" + rex2 + ")"
            filename = './grammars/texts/' + lang
            with open(filename) as file:
                for line in file:
                    if '\x0c' in line:
                        page = page + 1
                        continue
                    linenum = linenum + 1
                    glossline = re.findall(regex, line)# To ignorecase: add after comma in parenth: flags=re.IGNORECASE
                    for thing in glossline:
                        output = str(gloss) + '\t' + lang + '\t' + str(linenum) + '\t' + str(page) + '\t' + str(line) 
                        results.write(output)

## Exploring the data

### pandas

- Data analysis-software library for Python
- R users = 'dplyr'
- MS Excel for Python
- Uses dataframes for the manipulation and analysis of data


In [74]:
import pandas as pd

In [73]:
me_df = pd.read_csv("multiple_exponence.tsv",
                 sep='\t',
                 names=["Gloss", "File", "Line#", "Page#", "Match"])
 

# Building datasets

<center>
<img src="Images/typscience.jpg" width="1000"> 

### Types of data

https://wals.info/


https://www.smg.surrey.ac.uk/syncretism/


https://lexicalsplitsdb.surrey.ac.uk/

https://github.com/autotyp/autotyp-data/tree/0.1.0


https://zenodo.org/record/4898419#.Yas2mNBByUk


https://sacha.beniamine.net/dataset/beniamine-maiden-round-2019/


https://www.gerlingo.com/

Move away from *databases* and toward *datasets*

- Data set is a collection of data in a tabular format
- Query the data directly
- Generalisable across projects
- As close to language data as possible

# Data/Code Sharing

<center>
<img src="Images/typscience.jpg" width="1000"> 

## Why share your data and code?

1. Reproducibility
2. Journals require it
3. Learn a lot
4. Extends the impact of your project
5. Build a portfolio

## Data

‘FAIR Guiding Principles for scientific data management and stewardship’ were published in *Scientific Data 2006*.

https://www.go-fair.org/fair-principles/


**F**indable
- Will anyone else know that your data exists?
    - Solutions: put it in a standard repository, or at least a description of the data. Get a digital object identifier (DOI).
    
**A**ccessible
- Once someone knows that the data exists, can they get it?
    - Usually solved by being in a repository, but for non-open data, may require more procedures.

**I**nteroperable
-  your data in a format that can be used by others, like csv instead of PDF?

**R**eusable
- Is there a license allowing others to re-use?


### Sharing on Zenodo

Directly:
- Easy
- Good for finalised datasets that wont change

Github:
- A couple more steps
- Allows version control

https://zenodo.org/
https://sandbox.zenodo.org/login/
